In [20]:
import gzip,pickle
import numpy as np
import random,math,sys
import utility
import pandas as pd
import copy
from operator import itemgetter


In [5]:
def greedy_round_robin(m,n,R,l,T,V,U,F): 
    # greedy round robin allocation based on a specific ordering of customers (assuming the ordering is done in the relevance scoring matrix before passing it here)
    
    # creating empty allocations
    B={}
    for u in U:
        B[u]=[]
    
    # available number of copies of each producer
    Z={} # total availability
    P=range(n) # set of producers
    for p in P:
        Z[p]=l
    
    # allocating the producers to customers
    for t in range(1,R+1):
        print("GRR round number==============================",t)
        for i in range(m):
            if T==0:
                return B,F
            u=U[i]
            # choosing the p_ which is available and also in feasible set for the user
            possible=[(Z[p]>0)*(p in F[u])*V[u,p] for p in range(n)] 
            p_=np.argmax(possible) 
            
            if (Z[p_]>0) and (p_ in F[u]) and len(F[u])>0:
                B[u].append(p_)
                F[u] = list(F[u])
                F[u].remove(p_)
                Z[p_]=Z[p_]-1
                T=T-1
            else:
                return B,F
    # returning the allocation
    return B,F;


def FairRec(k,V,alpha):
    print("Start FairRec:")
    m=V.shape[0] # number of customers
    n=V.shape[1] # number of producers
    
    U=range(m) # list of customers
    P=range(n) # list of producers

    # Allocation set for each customer, initially it is set to empty set
    A={}
    for u in U:
        A[u]=[]

    # feasible set for each customer, initially it is set to P
    F={}
    for u in U:
        F[u]=P[:]
    #print(sum([len(F[u]) for u in U]))
   
    # number of copies of each producer
    l=int(alpha*m*k/(n+0.0))

    # R= number of rounds of allocation to be done in first GRR
    R=int(math.ceil((l*n)/(m+0.0)))  

    
    # total number of copies to be allocated
    T= l*n
       
    # first greedy round-robin allocation
    print("Start Greedy Round Robin first round:")
    [B,F1]=greedy_round_robin(m,n,R,l,T,V,U[:],F.copy())
    F={}
    F=F1.copy()
    print("GRR done")
    # adding the allocation
    for u in U:        
        A[u]=A[u][:]+B[u][:]
    
    # second phase
    # This phase is different from demo code in paper. Since no limit items for greedy-round-robin, its become this
    u_less=[] # customers allocated with <k products till now
    for u in A:
        if len(A[u])<k:
            u_less.append(u)

    # allocating every customer till k products
    for u in u_less:
        scores=V[u,:]
        new=scores.argsort()[-(k+k):][::-1]
        for p in new:
            if p not in A[u]:
                A[u].append(p)
            if len(A[u])==k:
                break

    return A;


In [34]:
V=np.loadtxt('ml-1m-6.csv',delimiter=',')
print("relevance scoring data loaded")

# size of recommendation
reco_size=int(20)

# fraction of MMS to be guaranteed to every producer
alpha=float(1)

# calling FairRec
A=FairRec(reco_size,V,alpha)

relevance scoring data loaded
Start FairRec:
Start Greedy Round Robin first round:
GRR round number============================== 1
GRR round number============================== 2
GRR round number============================== 3
GRR round number============================== 4
GRR round number============================== 5
GRR round number============================== 6
GRR round number============================== 7
GRR round number============================== 8
GRR round number============================== 9
GRR round number============================== 10
GRR round number============================== 11
GRR round number============================== 12
GRR round number============================== 13
GRR round number============================== 14
GRR done


In [66]:
train_df = pd.read_pickle(r'ml1m-6/training_df.pkl')
vali_df = pd.read_pickle(r'ml1m-6/testing_df.pkl')   # for validation
# vali_df = pickle.load(open('./' + dataname + '/testing_df.pkl'))  # for testing
key_genre = pd.read_pickle(r'ml1m-6/key_genre.pkl')
item_idd_genre_list = pd.read_pickle(r'ml1m-6/item_idd_genre_list.pkl')
genre_item_vector = pd.read_pickle(r'ml1m-6/genre_item_vector.pkl')
genre_count = pd.read_pickle(r'ml1m-6/genre_count.pkl')
user_genre_count = pd.read_pickle(r'ml1m-6/user_genre_count.pkl')

num_item = len(train_df['item_id'].unique())
num_user = len(train_df['user_id'].unique())
num_genre = len(key_genre)
top1 = 1
top2 = 5
top3 = 10
top4 = 15


In [67]:
item_genre_list = []
for u in range(num_item):
    gl = item_idd_genre_list[u]
    tmp = []
    for g in gl:
        if g in key_genre:
            tmp.append(g)
    item_genre_list.append(tmp)

item_genre = np.zeros((num_item, num_genre))
for i in range(num_item):
    gl = item_genre_list[i]
    for k in range(num_genre):
        if key_genre[k] in gl:
            item_genre[i, k] = 1.0

genre_count_mean_reciprocal = []

##there are six key_genre --> in the training dataset, count the number of movies for each genre
#genre_count = dictionary with number of movies for each keygrenre
for k in key_genre:
    genre_count_mean_reciprocal.append(1.0 / genre_count[k])
genre_count_mean_reciprocal = (np.array(genre_count_mean_reciprocal)).reshape((num_genre, 1))
genre_error_weight = np.dot(item_genre, genre_count_mean_reciprocal)

In [68]:
Q_tensor = (np.load('Q_tensor.npy'))
P_tensor = (np.load('P_tensor.npy'))
Rec = np.matmul(P_tensor, (Q_tensor).T)
Rec

array([[ 3.777458 ,  4.221402 ,  5.198299 , ..., -4.2804947, -2.578201 ,
        -2.816697 ],
       [-0.811322 ,  1.6305199,  1.3584685, ..., -1.1151903, -3.3449132,
        -4.613763 ],
       [ 2.7363856,  1.0603654,  5.1922445, ..., -3.727843 , -3.6910203,
        -3.445971 ],
       ...,
       [ 0.3022063,  2.698015 ,  1.7298616, ..., -1.4951588, -1.1846492,
        -1.3607007],
       [ 3.5729375,  6.708638 ,  4.714961 , ..., -4.493719 , -2.7313833,
        -2.9217112],
       [ 1.204689 ,  2.7632651,  1.7827291, ..., -3.1170988, -1.994886 ,
        -3.3972826]], dtype=float32)

In [69]:
Rec = copy.copy(Rec)

count1_dict = dict()
count5_dict = dict()
count10_dict = dict()
count15_dict = dict()
test_count = dict()
recall1_dict = dict()
recall5_dict = dict()
recall10_dict = dict()
recall15_dict = dict()
user_count_dict = dict()
num_user = Rec.shape[0]
num_item = Rec.shape[1]
top1_dict = dict()
top5_dict = dict()
top10_dict = dict()
top15_dict = dict()
avg_top1_dict = dict()
avg_top5_dict = dict()
avg_top10_dict = dict()
avg_top15_dict = dict()
tmp_top1_dict = dict()
tmp_top5_dict = dict()
tmp_top10_dict = dict()
tmp_top15_dict = dict()
genre_rank_count = dict()
rank_count = np.ones(num_item) * 1e-10

genre_to_be_rank = dict()

for k in key_genre:
    genre_rank_count[k] = np.zeros(num_item)
    top1_dict[k] = 0.0
    top5_dict[k] = 0.0
    top10_dict[k] = 0.0
    top15_dict[k] = 0.0
    avg_top1_dict[k] = 0.0
    avg_top5_dict[k] = 0.0
    avg_top10_dict[k] = 0.0
    avg_top15_dict[k] = 0.0
    tmp_top1_dict[k] = 0.0
    tmp_top5_dict[k] = 0.0
    tmp_top10_dict[k] = 0.0
    tmp_top15_dict[k] = 0.0

    recall1_dict[k] = 0.0
    recall5_dict[k] = 0.0
    recall10_dict[k] = 0.0
    recall15_dict[k] = 0.0
    user_count_dict[k] = 0.0
    count1_dict[k] = 0.0
    count5_dict[k] = 0.0
    count10_dict[k] = 0.0
    count15_dict[k] = 0.0
    genre_to_be_rank[k] = 0.0
    test_count[k] = 0.0

for u in range(num_user):
    like_item = (train_df.loc[train_df['user_id'] == u, 'item_id']).tolist()
    Rec[u, like_item] = -100000.0

In [70]:
for i in A.keys():  # iterate each user
    u_test = (vali_df.loc[vali_df['user_id'] == u, 'item_id']).tolist()
    u_pred = Rec[u, :]

    top15_item_idx_no_train = A[i][:-5]
    top15 = (np.array([top15_item_idx_no_train, u_pred[top15_item_idx_no_train]])).T
    top15 = sorted(top15, key=itemgetter(1), reverse=True)

    # calculate the recall for different genres
    if not len(u_test) == 0:
        recall_1_tmp_dict, recall_5_tmp_dict, recall_10_tmp_dict, recall_15_tmp_dict, \
        count_1_tmp_dict, count_5_tmp_dict, count_10_tmp_dict, count_15_tmp_dict, test_tmp_dict\
            = utility.user_recall(top15, u_test, item_genre_list, key_genre)
        for k in key_genre:
            count1_dict[k] += count_1_tmp_dict[k]
            count5_dict[k] += count_5_tmp_dict[k]
            count10_dict[k] += count_10_tmp_dict[k]
            count15_dict[k] += count_15_tmp_dict[k]
            test_count[k] += test_tmp_dict[k]
            if recall_1_tmp_dict[k] == -1:
                continue
            recall1_dict[k] += recall_1_tmp_dict[k]
            recall5_dict[k] += recall_5_tmp_dict[k]
            recall10_dict[k] += recall_10_tmp_dict[k]
            recall15_dict[k] += recall_15_tmp_dict[k]
            user_count_dict[k] += 1.0

    # calculate ranking probability
    rank = 1
    for r in top15:
        gl = item_idd_genre_list[int(r[0])]
        for g in gl:
            if g in key_genre:
                genre_rank_count[g][rank - 1] += 1.0
                rank_count[rank - 1] += 1.0
                if rank <= top4:
                    tmp_top15_dict[g] += 1.0
                    if rank <= top3:
                        tmp_top10_dict[g] += 1.0
                        if rank <= top2:
                            tmp_top5_dict[g] += 1.0
                            if rank <= top1:
                                tmp_top1_dict[g] += 1.0
        rank += 1
    for k in key_genre:
        top1_dict[k] += tmp_top1_dict[k]
        top5_dict[k] += tmp_top5_dict[k]
        top10_dict[k] += tmp_top10_dict[k]
        top15_dict[k] += tmp_top15_dict[k]
        avg_top1_dict[k] += (1.0 * tmp_top1_dict[k] / user_genre_count[u][k])
        avg_top5_dict[k] += (1.0 * tmp_top5_dict[k] / user_genre_count[u][k])
        avg_top10_dict[k] += (1.0 * tmp_top10_dict[k] / user_genre_count[u][k])
        avg_top15_dict[k] += (1.0 * tmp_top15_dict[k] / user_genre_count[u][k])
        tmp_top1_dict[k] = 0.0
        tmp_top5_dict[k] = 0.0
        tmp_top10_dict[k] = 0.0
        tmp_top15_dict[k] = 0.0
        genre_to_be_rank[k] += user_genre_count[u][k]

In [71]:
def relative_std(dictionary):
    tmp = []
    for key, value in sorted(dictionary.items(), key = lambda x: x[0]):
        tmp.append(value)
    rstd = np.std(tmp) / (np.mean(tmp) + 1e-10)
    return rstd

In [72]:
for k in key_genre:
    count1_dict[k] /= test_count[k]
    count5_dict[k] /= test_count[k]
    count10_dict[k] /= test_count[k]
    count15_dict[k] /= test_count[k]
    recall1_dict[k] /= user_count_dict[k]
    recall5_dict[k] /= user_count_dict[k]
    recall10_dict[k] /= user_count_dict[k]
    recall15_dict[k] /= user_count_dict[k]
print('')
print('#' * 100)
print('# System-level Recall:')
print('# \t\t\tRecall@%d\tRecall@%d\tRecall@%d\tRecall@%d' % (top1, top2, top3, top4))
for k in key_genre:
    print('# ' + k + '\t\t%.5f\t\t%.5f\t\t%.5f\t\t%.5f' % (count1_dict[k], count5_dict[k], count10_dict[k], count15_dict[k]))
recall1_std = relative_std(count1_dict)
recall5_std = relative_std(count5_dict)
recall10_std = relative_std(count10_dict)
recall15_std = relative_std(count15_dict)
print('# relative std\t\t%.5f\t\t%.5f\t\t%.5f\t\t%.5f' % (recall1_std, recall5_std, recall10_std, recall15_std))
print('#' * 100)

print('# User-level Recall:')
print('# \t\t\tRecall@%d\tRecall@%d\tRecall@%d\tRecall@%d' % (top1, top2, top3, top4))
for k in key_genre:
    print('# ' + k + '\t\t%.5f\t\t%.5f\t\t%.5f\t\t%.5f' % (
        recall1_dict[k], recall5_dict[k], recall10_dict[k], recall15_dict[k]))
recall1_std_user = relative_std(recall1_dict)
recall5_std_user = relative_std(recall5_dict)
recall10_std_user = relative_std(recall10_dict)
recall15_std_user = relative_std(recall15_dict)
print('# relative std\t\t%.5f\t\t%.5f\t\t%.5f\t\t%.5f' % (recall1_std_user, recall5_std_user, recall10_std_user, recall15_std_user))
print('#' * 100)

# calculate the average genre ranking probability across users, and calculate system-level ranking probability
for k in key_genre:
    avg_top1_dict[k] /= num_user
    avg_top5_dict[k] /= num_user
    avg_top10_dict[k] /= num_user
    avg_top15_dict[k] /= num_user

    top1_dict[k] /= genre_to_be_rank[k]
    top5_dict[k] /= genre_to_be_rank[k]
    top10_dict[k] /= genre_to_be_rank[k]
    top15_dict[k] /= genre_to_be_rank[k]

print('# System-level top ranking probability:')
print('# \t\t\t@%d\t\t@%d\t\t@%d\t\t@%d' % (top1, top2, top3, top4))
for k in key_genre:
    print('# ' + k + '\t\t%.5f\t\t%.5f\t\t%.5f\t\t%.5f' % (top1_dict[k], top5_dict[k], top10_dict[k], top15_dict[k]))
top1_std = relative_std(top1_dict)
top5_std = relative_std(top5_dict)
top10_std = relative_std(top10_dict)
top15_std = relative_std(top15_dict)
print('# relative std\t\t%.5f\t\t%.5f\t\t%.5f\t\t%.5f' % (top1_std, top5_std, top10_std, top15_std))
print('#' * 100)

print('# User-level top ranking probability:')
print('# \t\t\t@%d\t\t@%d\t\t@%d\t\t@%d' % (top1, top2, top3, top4))
for k in key_genre:
    print('# ' + k + '\t\t%.5f\t\t%.5f\t\t%.5f\t\t%.5f' % (avg_top1_dict[k], avg_top5_dict[k], avg_top10_dict[k], avg_top15_dict[k]))
top1_std_user = relative_std(avg_top1_dict)
top5_std_user = relative_std(avg_top5_dict)
top10_std_user = relative_std(avg_top10_dict)
top15_std_user = relative_std(avg_top15_dict)
print('# relative std\t\t%.5f\t\t%.5f\t\t%.5f\t\t%.5f' % (top1_std_user, top5_std_user, top10_std_user, top15_std_user))
print('#' * 100)


####################################################################################################
# System-level Recall:
# 			Recall@1	Recall@5	Recall@10	Recall@15
# Sci-Fi		0.01052		0.02477		0.02778		0.02781
# Horror		0.00977		0.01822		0.01972		0.01972
# Crime		0.01359		0.02005		0.02005		0.02005
# Adventure		0.01549		0.03392		0.03487		0.03487
# Children's		0.01077		0.02038		0.02046		0.02046
# Romance		0.00736		0.01399		0.01512		0.01512
# relative std		0.23378		0.28570		0.28168		0.28177
####################################################################################################
# User-level Recall:
# 			Recall@1	Recall@5	Recall@10	Recall@15
# Sci-Fi		0.01052		0.02477		0.02778		0.02781
# Horror		0.00977		0.01822		0.01972		0.01972
# Crime		0.01359		0.02005		0.02005		0.02005
# Adventure		0.01549		0.03392		0.03487		0.03487
# Children's		0.01077		0.02038		0.02046		0.02046
# Romance		0.00736		0.01399		0.01512		0.01512
# relative std		0.23378		0.28570		0.28168		0.28177
###########